In [1]:
# !pip install import_ipynb --quiet
# !pip install anvil-uplink --quiet
# !pip install yfinance --quiet
# !pip install pandas_ta --quiet
# !pip install ipynb --quiet
# !pip install rpyc --quiet
# !pip install stable-baselines3 --quiet
# !pip install aspectlib

In [2]:
# !git clone https://github.com/gmshroff/algostrats.git

In [3]:
# %cd algostrats

Uncomment below if on Colab and using datasets from Kaggle

In [4]:
# from google.colab import files
# uploaded=files.upload()

In [5]:
# !mkdir /root/.kaggle
# !mv ./kaggle.json /root/.kaggle/.
# !chmod 600 /root/.kaggle/kaggle.json

In [6]:
# %mkdir data
# %cd data
# !kaggle datasets download -d gmshroff/marketdatafivemin
# %cd ../algostrats

In [8]:
colab=False
script=False
if not colab: DATAPATH='~/DataLocal/algo_fin_new/five_min_data/'
elif colab: DATAPATH='../algodata/'

Need to import algorithms from stable-baselines3

In [9]:
from stable_baselines3 import PPO,A2C,DQN
from stable_baselines3.common.vec_env import StackedObservations
from stable_baselines3.common.monitor import Monitor as Mon

In [10]:
import warnings
warnings.simplefilter("ignore")

In [11]:
import import_ipynb
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from datetime import datetime as dt
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import pickle
from threading import Thread
import threading
from IPython import display
import time,getopt,sys,os

In [12]:
from feeds import BackFeed,DataFeed
from featfuncs import feat_aug,add_addl_features_feed,add_ta_features_feed,add_sym_feature_feed
from featfuncs import add_global_indices_feed

importing Jupyter notebook from feeds.ipynb
importing Jupyter notebook from synfeed.ipynb
importing Jupyter notebook from india_calendar.ipynb
importing Jupyter notebook from featfuncs.ipynb


In [13]:
from feed_env import Episode
import aspectlib

importing Jupyter notebook from feed_env.ipynb


### Trading strategies as agents

RL(++)StratAgents imported from ruleagents<br>

In [27]:
from rlagents import RLStratAgentDyn, RLStratAgentDynFeatures

### Strategy Testing for RLStratAgent using BackTest

In [15]:
from backtest import Backtest
from feeds import BackFeed,DataFeed
from validation import Validate

importing Jupyter notebook from backtest.ipynb
importing Jupyter notebook from validation.ipynb


<b>Configuration for training RL model<b>

In [16]:
algorithm=PPO
synthetic=False #use synthetic data
simple='sinewave'
nw,nd=4,5 #for BackFeed

In [17]:
if script:
    try:
        opts,args = getopt.getopt(sys.argv[1:],"hl:f:d:m:s:w:t:p:u:",["load=","feed=","datafile=","modelname=","synthetic","weeks","training_steps","deploy","use_alt_data"])
    except getopt.GetoptError:
        print('rlagents_train.py -l <load:True/False> -f <scan:back/data> -d <datafile> -m <modelname> -s <synthetic> -w <weeks> -t <training_steps> -p <deploy>')
        sys.exit(2)
    load,feed,modelname=False,'back',''
    training_steps=50000 # if less then n_steps then n_steps is used
    deploy=True
    date=datetime.today().strftime('%d-%b-%Y')
    use_alt_data=False
    for opt, arg in opts:
        if opt == "-h":
            print('rlagents_train.py -l <load:True/False> -f <scan:back/data> -d <datafile> -m <modelname> -s <synthetic> -w <weeks> -t <training_steps> -p <deploy> -u <use_alt_data>')
            sys.exit()
        elif opt in ("-l", "--load"):
            load = (lambda x: True if x=='True' else False)(arg)
        elif opt in ("-f", "--feed"):
            feed = (lambda x: 'data' if x=='data' else 'back')(arg)
        elif opt in ("-d", "--datafile"):
            datafile = arg.split('/')[-1]
        elif opt in ("-m", "--modelname"):
            modelname = arg
        elif opt in ("-s", "--synthetic"):
            synthetic = (lambda x: True if x=='True' else False)(arg)
        elif opt in ("-w", "--weeks"):
            nw = int(arg)
        elif opt in ("-t", "--training_steps"):
            training_steps=int(arg)
        elif opt in ("-p", "--deploy"):
            deploy = (lambda x: True if x=='True' else False)(arg)
        elif opt in ("-u", "--use_alt_data"):
            use_alt_data = (lambda x: True if x=='True' else False)(arg)
    if len(opts)==0: 
        print('rlagents_train.py -l <load:True/False> -f <scan:back/data> -d <datafile> -m <modelname> -s <synthetic> -w <weeks> -t <training_steps> -p <deploy> -u <use_alt_data>')
        sys.exit()
    print(f"load:{load},feed:{feed},datafile:{datafile},modelname:{modelname},synthetic:{synthetic},weeks:{nw},training_steps:{training_steps},deploy:{deploy},use_alt_data:{use_alt_data}")
    loadfeed=load
    if feed=='data': datafeed=True
    else: datafeed=False

In [18]:
if not script:
    loadfeed=True
    datafeed=True
    datafile='alldata.csv'
    # modelname='RLA0.pth' # replace with modelname if model to be loaded
    modelname='RLC0_all.pth'
    date=datetime.today().strftime('%d-%b-%Y')
    deploy=True

In [19]:
n_steps=2048 # reduce for debugging only else 2048

In [20]:
def stringify(x):
    return pd.to_datetime(x['Datetime']).strftime('%d-%b-%Y')

In [21]:
import pickle
if not loadfeed and not datafeed:
    data=pd.read_csv('./capvolfiltered.csv')
    tickers=list(data.iloc[0:50]['ticker'].values)
    print('Creating feed')
    feed=BackFeed(tickers=tickers,nd=nd,nw=nw,interval='5m',synthetic=synthetic)
    print('Processing feed')
    add_addl_features_feed(feed,tickers=feed.tickers)
    add_sym_feature_feed(feed,tickers=feed.tickers)
    add_global_indices_feed(feed)
    if colab: 
        with open('/tmp/btfeed.pickle','wb') as f: pickle.dump(feed,f)
    else: 
        with open('../../temp_data/btfeed.pickle','wb') as f: pickle.dump(feed,f)
elif loadfeed and not datafeed:
    if colab: 
        with open('/tmp/btfeed.pickle','rb') as f: feed=pickle.load(f)
    else: 
        with open('../../temp_data/btfeed.pickle','rb') as f: feed=pickle.load(f)

In [22]:
if not loadfeed and datafeed:
    #DATAFILE=DATAPATH+'augdata_'+date+'_5m.csv'
    DATAFILE=DATAPATH+datafile
    print('Reading datafile')
    df=pd.read_csv(DATAFILE)
    if 'Date' not in df.columns: 
        print('Adding Date')
        df['Date']=df.apply(stringify,axis=1)
    print('Creating feed')
    feed=DataFeed(tickers=list(df.ticker.unique()[0:10]),dfgiven=True,df=df)
    print('Processing feed')
    add_addl_features_feed(feed,tickers=feed.tickers)
    add_sym_feature_feed(feed,tickers=feed.tickers)
    add_global_indices_feed(feed)
    if not colab: 
        with open('../../temp_data/btdatafeed.pickle','wb') as f: pickle.dump(feed,f)
    elif colab: 
        with open('/tmp/btdatafeed.pickle','wb') as f: pickle.dump(feed,f)
elif loadfeed and datafeed:
    if not colab: 
        with open('../algodata/realdata/datafeed_alldata_True_True.pkl','rb') as f: feed=pickle.load(f)
    elif colab:
        with open('/tmp/btdatafeed.pickle','rb') as f: feed=pickle.load(f)

In [24]:
for t in feed.ndata:
    for d in feed.ndata[t]:
        if feed.ndata[t][d].isnull().values.any(): 
            feed.ndata[t][d]=feed.ndata[t][d].fillna(1)
            # print(t,d)
        if feed.ndata[t][d].isin([-np.inf,np.inf]).values.any():
            feed.ndata[t][d]=feed.ndata[t][d].replace([np.inf, -np.inf],1)
            # print(t,d)

In [25]:
# from math import isnan
# for d in feed.gdata:
#     for g in feed.gdata[d]:
#          for k in g: 
#                 if isnan(g[k]): g[k]=1

In [26]:
def get_alt_data_live():
    aD={'gdata':feed.gdata}
    return aD

In [28]:
use_alt_data=False
agent=RLStratAgentDynFeatures(algorithm,monclass=Mon,soclass=StackedObservations,verbose=1,
                   metarl=True,myargs=(n_steps,use_alt_data,5))
agent.use_memory=True #depends on whether RL algorithm uses memory for state computation
agent.debug=False
if use_alt_data: agent.set_alt_data(alt_data_func=get_alt_data_live)

Using cpu device
Wrapping the env in a DummyVecEnv.


In [29]:
agent.training=False

In [30]:
if modelname and os.path.exists('./saved_models/'+modelname): 
    print('Loading model')
    agent.load_model(filepath='./saved_models/'+modelname)

Loading model


In [31]:
@aspectlib.Aspect
def my_decorator(*args, **kwargs):
    # print("Got called with args: %s kwargs: %s" % (args, kwargs))
    # result = yield
    # print(" ... and the result is: %s" % (result,))
    state,rew,done,exit_type = yield
    # args[0].policy.logL+=[(state.keys(),rew,done,exit_type)]
    args[0].policy.reward((rew,done,{'exit_type':exit_type}))
    return state,rew,done,exit_type

In [32]:
aspectlib.weave(Episode, my_decorator, methods='env_step')

In [33]:
bt=Backtest(feed,tickers=feed.tickers,add_features=False,target=5,stop=5,txcost=0.001,
            loc_exit=True,scan=True,topk=5,deploy=deploy,save_dfs=False,
            save_func=None)

In [ ]:
def compute_avg_episode_rew(bt):
    i,tot=0,0
    for t in bt.results:
        for d in bt.results[t]:
            for r in bt.results[t][d]['rew']:
                tot+=r[1]
                i+=1
    return tot/i,i,tot

In [36]:
agent.data_cols=['row_num', 'Open_n', 'High_n', 'Low_n', 'Close_n', 'Volume_n', 'SMA_10', 'SMA_20', 'VOL_SMA_20', 'RSI_14', 'BBL_5_2.0', 'BBM_5_2.0', 'BBU_5_2.0', 'BBB_5_2.0', 'BBP_5_2.0', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'VWAP_D', 'MOM_30', 'CMO_14', 'CMO_14_slope_1_polarity', 'Open_ma_1-Open_ma_10_val', 'Volume_ma_5_val', 'BBB_5_2.0_changelen_val', 'Open-Close_ma_1-Open-Close_ma_5_val', 'Volume_ma_20-Volume_ma_50_changelen_val', 'Open_n_ma_10_slope_5_changelen_polarity', 'Close_slope_1_polarity', 'RSI_14_ma_1_val', 'CMO_14_changelen_val', 'High_n-Low_n_ma_50_val', 'Open-Close_polarity', 'Open-High_ma_1-Open-High_ma_50_val', 'Open-Close_ma_1-Open-Close_ma_20_val', 'Close_n_ma_1-Close_n_ma_50_changelen_polarity', 'High_n_ma_1-High_n_ma_20_changelen_polarity', 'Open-High_ma_1-Open-High_ma_50_polarity']

In [37]:
bt.run_all(tickers=feed.tickers,model=agent,verbose=True)

NELCAST.NS 03-Feb-2022 0 45 -1 2.0105421686747027 45
NELCAST.NS 03-Feb-2022 45 46 -1 -0.03858520882261844 46
NELCAST.NS 04-Feb-2022 0 9 -1 -2.9530699777900895 9
NELCAST.NS 04-Feb-2022 9 48 -1 1.9792284866468912 48
NELCAST.NS 04-Feb-2022 48 48 -1 -0.1 48
NELCAST.NS 07-Feb-2022 0 12 -1 1.9966486367628469 12
NELCAST.NS 07-Feb-2022 12 35 -1 0.9382532083955839 35
NELCAST.NS 08-Feb-2022 0 31 -1 1.1222221994509995 31
NELCAST.NS 09-Feb-2022 0 30 -1 0.27051578470367643 30
NELCAST.NS 10-Feb-2022 0 18 -1 2.0626543209876527 18
NELCAST.NS 10-Feb-2022 18 41 -1 -0.3508733553062366 41
NELCAST.NS 11-Feb-2022 0 37 -1 -0.28768940925599035 37
NELCAST.NS 14-Feb-2022 0 38 -1 -0.1 38
NELCAST.NS 15-Feb-2022 0 6 -1 1.9183976098216937 6
NELCAST.NS 15-Feb-2022 6 12 -1 2.4267921620588178 12
NELCAST.NS 15-Feb-2022 12 17 -1 -2.7944817197455416 17
NELCAST.NS 15-Feb-2022 17 44 -1 -3.111514907408386 44
NELCAST.NS 15-Feb-2022 44 49 -1 -0.22767662072323683 49
NELCAST.NS 16-Feb-2022 0 32 -1 1.9260671066267931 32
NELCAST.

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
avg,n_ep,tot=compute_avg_episode_rew(bt) # Avg episode reward (not day)
print(f"Average reward {avg} across {n_ep} episodes; total {tot}")

In [ ]:
print(f"Average returns per day:{tot/len(feed.dates)}")

In [ ]:
maxdd=min([bt.results[t][d]['tot'] for t in bt.results for d in bt.results[t]])
print(f"Max daily drawdown:{maxdd}")

In [ ]:
maxdp=max([bt.results[t][d]['tot'] for t in bt.results for d in bt.results[t]])
print(f"Max daily profit:{maxdp}")

In [ ]:
# sum([bt.returns[t] for t in bt.returns])

## Trading Plots

In [ ]:
import plotly.graph_objects as go

In [ ]:
import plotly.express as px

In [ ]:
def annotate_action(rew,act,df):
    if rew[1]>=0:color='Green'
    else: color='Red'
    if act[0]==1:text='Buy'
    elif act[0]==-1:text='Sell'
    ann=dict(font=dict(color=color,size=15),x=df.index[rew[0]],y=df.iloc[rew[0]]['Close'],
             showarrow=True,text=text)
    return ann

In [ ]:
def annotate_exit(rew,act,anns,df):
    if rew[1]>=0:color='Green'
    else: color='Red'
    X=[a['x'] for a in anns if a is not None]
    if df.index[rew[2]] in X: 
        idx=X.index(df.index[rew[2]])
        anns[idx]['text']='Ex&'+anns[idx]['text']
    else:
        anns+=[dict(font=dict(color=color,size=15),x=df.index[rew[2]],y=df.iloc[rew[2]]['Close'],
                    showarrow=True,text='Exit')]

In [ ]:
def plot_ticker_date(bt,ticker,date):
    global fig
    df=feed.ndata[ticker][date]
    df=df.loc[df['Date']==date]
    fig = go.Figure(data=
        [go.Candlestick(x = df.index,
                        open  = df["Open"],
                        high  = df["High"],
                        low   = df["Low"],
                        close = df["Close"])]
    )
    reward=np.round(bt.results[ticker][date]["tot"],2)
    fig.update_layout(
        title=f'{ticker} on {date} return {reward}',
        yaxis_title="Price"
    )
    anns=[]
    for r,a in zip(bt.results[ticker][date]['rew'],bt.results[ticker][date]['acts']):
        anns+=[annotate_action(r,a,df)]
    for r,a in zip(bt.results[ticker][date]['rew'],bt.results[ticker][date]['acts']):
        anns+=[annotate_exit(r,a,anns,df)]
    for a in anns: 
        if a is not None: fig.add_annotation(a)
    fig.show()
    return fig

In [ ]:
def combine_plotly_figs_to_html(plotly_figs, html_fname, include_plotlyjs='cdn', 
                                separator=None, auto_open=False):
    with open(html_fname, 'w') as f:
        f.write(plotly_figs[0].to_html(include_plotlyjs=include_plotlyjs))
        for fig in plotly_figs[1:]:
            if separator:
                f.write(separator)
            f.write(fig.to_html(full_html=False, include_plotlyjs=False))

    if auto_open:
        import pathlib, webbrowser
        uri = pathlib.Path(html_fname).absolute().as_uri()
        webbrowser.open(uri)

In [ ]:
if not script:
    figs=[]
    for t in bt.results:
        for d in bt.results[t]:
            figs+=[plot_ticker_date(bt,t,d)]

In [ ]:
if not script: combine_plotly_figs_to_html(figs,'../../temp_data/charts.html',auto_open=True)